In [1]:
import pandas as pd
import re
import openpyxl

In [43]:
tar_date = "2021-06-30"
bef_date = "2021-05-31"
pref = "D:/디지털혁신팀/태블로/HR현황/HR현황_FINAL"

In [44]:
hr = pd.read_excel(f"{pref}/직원명단_{tar_date.replace('-', '')[2:]}.xlsx")
# hr["본부명"] = hr["본부명"].str.replace("수주본부", "개발영업본부", regex=False)

## 인사 현황 데이터 전처리

### 건설본부만 뽑기

In [45]:
# hq_const = hr[hr["본부명"]=="건설본부"]
# hq_const.to_excel(f"{pref}/직원명단_210630_건설본부만.xlsx", encoding="euc-kr", index=False)

# hq_no_const = hr[hr["본부명"]!="건설본부"]
# hq_no_const.to_excel(f"{pref}/직원명단_210630_건설본부 제외.xlsx", encoding="euc-kr", index=False)

### 잘못된 데이터 바로잡기

In [46]:
hr.loc[hr["사번"]==1101290, "본부명"] = "개발영업본부"

### 이하나M 회신 제외 인원 반영

In [47]:
num_list = [6319, 4001308, 4100189, 4100190, 4100194, 4100196, 4100248, 4100251, 4100262, 4374, 4487]
hr.loc[(hr["사번"].isin(num_list)), "제외여부"] = "본부인원에서만 제외"

### 제외 로직 적용

In [48]:
# 본사, 지사, 현장 설정
hr[["직급", "직무", "하위그룹"]] = hr[["직급", "직무", "하위그룹"]].astype("str")
hr["코스트센터 분류"] = hr.apply(lambda x:"지사" if ("사업소" in x["조직명"]) or ("베트남지사" in x["조직명"]) else ("본사" if re.search("^\d", x["코스트센터"]) else "현장"), axis=1)
hr.loc[hr["코스트센터 분류"]=="현장", "본부명"] = "현장"
hr["코스트센터 본부명"] = hr["본부명"]
hr["코스트센터 팀명"] = hr["원소속명"]

# "본부인원에서만 제외" 설정
hr["제외여부"] = hr.apply(lambda x:"본부인원에서만 제외" if (x["본부명"]=="경영본부") & ((x["직급"]=="인턴") | (x["직무"]=="비상계획") | (x["직무"]=="축구협") | (x["직무"]=="비서") | (x["직무"]=="미화") | (x["직무"]=="기사")) else x["제외여부"], axis=1)
hr["제외여부"] = hr.apply(lambda x:"본부인원에서만 제외" if "대기" in x["조직명"] else x["제외여부"], axis=1)
hr["제외여부"] = hr.apply(lambda x:"본부인원에서만 제외" if x["직무"]=="휴직" else x["제외여부"], axis=1)

# "제외" 설정
hr.loc[(hr["하위그룹"].isin(["CS외주", "정보기술외주", "촉탁직", "파견직"])) | (hr["재직여부"]=="퇴직"), "제외여부"] = "제외"

hr = hr[["사번", "성명", "EMAIL", "본부명", "원소속코드", "원소속명", "조직코드", "조직명", "직급", "구분", "직종", "성별", "하위그룹", "재직여부", "퇴사일", "직무", "제외여부", "코스트센터", "코스트센터내역", "코스트센터 분류", "코스트센터 본부명", "코스트센터 팀명"]]

In [49]:
hr.to_excel(f"{pref}/직원명단_{tar_date.replace('-', '')[2:]}_전처리 최종.xlsx", index=False)

## 직급별 인원 파일 만들기

In [50]:
hr.loc[hr["코스트센터 본부명"]=="회장직속", "코스트센터 본부명"] = "사장직속"
hr.loc[hr["코스트센터 본부명"]=="부회장직속", "코스트센터 본부명"] = "사장직속"

cls2cnt = hr[hr["제외여부"]!="제외"]["코스트센터 분류"].value_counts()
hq2cnt = hr[(hr["제외여부"]!="제외") & (hr["제외여부"]!="본부인원에서만 제외")]["코스트센터 본부명"].value_counts()
print(cls2cnt, end="\n\n")
print(hq2cnt)

현장    1062
본사     463
지사      89
Name: 코스트센터 분류, dtype: int64

현장        1062
개발영업본부     263
건설본부       159
경영본부        49
미래혁신본부      41
사장직속        21
Name: 코스트센터 본부명, dtype: int64


In [51]:
wb = openpyxl.load_workbook(f"{pref}/인원 추이_전처리_/21.05/{bef_date.replace('-', '')[2:]}.xlsx")

In [52]:
ws = wb["근무지별 추이(5년치)"]
i = 2
for idx, value in cls2cnt.items():
    ws[f"A{i}"] = tar_date
    if idx == "지사":
        ws[f"B{i}"] = f"{idx}외"
    else:
        ws[f"B{i}"] = idx
    ws[f"C{i}"] = value
    i += 1

ws = wb["본부별 월별 추이(2021년)"]
for idx, value in hq2cnt.items():
    ws.insert_rows(2)
    ws["A2"] = tar_date
    ws["B2"] = idx
    ws["C2"] = value

In [53]:
wb.save(f"{pref}/인원 추이_전처리_{tar_date.replace('-', '')[2:]}.xlsx")

## 발령 이력

In [23]:
# appo = pd.read_csv("D:/디지털혁신팀/태블로/HR 분석/발령이력_18년이후_송부.csv")

# appo = appo.rename({"성명":"name", "직급(위)":"rank", "발령시작일":"start", "발령종료일":"end"}, axis=1)

# appo["start"] = appo["start"].apply(lambda x:pd.to_datetime(x, format="%Y%m%d"))
# appo["end"] = appo["end"].apply(lambda x:pd.to_datetime(x, format="%Y%m%d") if x != 99991231 else "now")
# appo["end"] = appo["end"].dt.normalize()

In [24]:
# ref_date = "2021-03-31"
# df = appo[(appo["start"]<=ref_date) & (appo["end"]>=ref_date)]